<a href="https://colab.research.google.com/github/MrAdithya21/Real-Time-Reddit-Sentiment-Analysis/blob/main/store_in_azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
from azure.storage.queue import QueueClient
from azure.storage.blob import BlobServiceClient

# Load Azure connection strings from environment variables
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
QUEUE_NAME = "reddit-comments"
BLOB_CONTAINER_NAME = "reddit-data"

# Azure Storage Queue Connection
queue_client = QueueClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING, QUEUE_NAME)

# Azure Blob Storage Connection
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(BLOB_CONTAINER_NAME)

# Retrieve Messages from Queue & Store in Blob
messages = queue_client.receive_messages(messages_per_page=10)

for msg in messages:
    blob_name = f"comment-{msg.id}.json"
    blob_client = container_client.get_blob_client(blob_name)
    blob_client.upload_blob(msg.content, overwrite=True)
    print(f"Stored {blob_name} in Blob Storage.")
    queue_client.delete_message(msg)  # Remove message after processing
